# Load data

In [1]:
import pandas as pd

train = pd.read_parquet('../input/train.parquet')

features = ['ip', 'app', 'device', 'os', 'channel']
target = ['is_attributed']

X = train[features]
y = train[target]

# Train

In [2]:
import keras
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping

from keras.models import Sequential
from keras.layers import Activation, Dropout, Dense, BatchNormalization
from keras.layers import MaxPooling1D, Lambda, Convolution1D, Flatten, SpatialDropout1D, AlphaDropout
from keras_tqdm import TQDMNotebookCallback
from keras.layers.merge import Concatenate

from keras.optimizers import Adam, RMSprop, Adamax, Adagrad, Nadam, SGD
from keras.activations import elu, relu, tanh, sigmoid
from keras.constraints import maxnorm
from keras.initializers import lecun_uniform

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def create_network(
    input_dim,
    n_dense=6,
    dense_units=16,
    activation='selu',
    dropout=AlphaDropout,
    dropout_rate=0.1,
    kernel_initializer='lecun_normal',
    optimizer='adam',
    num_classes=1,
    loss='binary_crossentropy'
    ):
    """Generic function to create a fully-connected neural network.
    # Arguments
        n_dense: int > 0. Number of dense layers.
        dense_units: int > 0. Number of dense units per layer.
        dropout: keras.layers.Layer. A dropout layer to apply.
        dropout_rate: 0 <= float <= 1. The rate of dropout.
        kernel_initializer: str. The initializer for the weights.
        optimizer: str/keras.optimizers.Optimizer. The optimizer to use.
        num_classes: int > 0. The number of classes to predict.
        max_words: int > 0. The maximum number of words per data point.
    # Returns
        A Keras model instance (compiled).
    """
    model = Sequential()
    model.add(Dense(dense_units, input_dim=input_dim,
                    kernel_initializer=kernel_initializer))
    model.add(Activation(activation))
    model.add(dropout(dropout_rate))

    for i in range(n_dense - 1):
        model.add(Dense(dense_units, kernel_initializer=kernel_initializer))
        model.add(Activation(activation))
        model.add(dropout(dropout_rate))

    model.add(Dense(num_classes))
    model.add(Activation('sigmoid'))
    model.compile(loss=loss,
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

In [4]:
network_selu = {
    'n_dense': 64,
    'dense_units': 32,
    'activation': 'selu',
    'dropout': AlphaDropout,
    'dropout_rate': 0.05,
    'kernel_initializer': 'lecun_normal',
    'optimizer': Adam(1e-5)
}

model = create_network(X.shape[1], **network_selu)

In [5]:
model.fit(
    X, y,
    batch_size= 4 * 4096,
    epochs=25,
    validation_split=0.05,
    verbose=0,
    callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)] 
)

HBox(children=(IntProgress(value=0, description='Training', max=25), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 0', max=175658695), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 1', max=175658695), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 2', max=175658695), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 3', max=175658695), HTML(value='')))

...

HBox(children=(IntProgress(value=0, description='Epoch 8', max=175658695), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 9', max=175658695), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 10', max=175658695), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 11', max=175658695), HTML(value='')))

KeyboardInterrupt: 

# Submission

In [ ]:
import gc

del train
gc.collect()

In [ ]:
test = pd.read_parquet('../input/test.parquet')

test_cols = ['ip', 'app', 'device', 'os', 'channel']
X_test = test[test_cols]

In [ ]:
test.head()

In [ ]:
submission = pd.DataFrame()
submission['click_id'] = test['click_id']
submission['is_attributed'] = model.predict(X_test, 128 * 4096)

In [ ]:
submission.head()

In [ ]:
submission.shape

In [ ]:
submission.to_csv('../submissions/selu_net_v3.csv', index=False)